In [1]:
from IPython.display import Markdown, display
from LLMResponse import LLMResponse

In [2]:
bot = LLMResponse()

items : 83, colours : 24, materials : 23


In [3]:
# 답변 생성
response = bot.GetLLMResponse(
    userInput = "겨울 맑은 날 출근 코디",
    max_scroll_count = 5,
    model_name = "openai:gpt-4.1-nano",
    max_tokens = 2048
)

# 아이템 언급 수 상위 5개 출력
display(response[1])
print("=" * 20)

# 색상 언급 수 상위 5개 출력
display(response[2])
print("=" * 20)

# 재질 언급 수 상위 5개 출력
display(response[3])
print("=" * 20)

# LLM 답변 출력
display(Markdown(response[0]))

니트     550
코트     142
패딩     132
자켓     126
운동화    108
Name: items, dtype: int64

화이트    125
그레이    109
블랙      70
크림      47
브라운     42
Name: colours, dtype: int64

울     460
퍼     187
면     122
다운     48
가죽     47
Name: materials, dtype: int64

```검색 키워드 >> 겨울 맑은 날 출근 코디```

# 겨울 맑은 날 출근 코디 추천

네이버 블로그 내용을 종합하면, 겨울철 출근 패션은 따뜻함과 세련됨을 동시에 고려하는 것이 중요합니다. 특히, 니트, 코트, 자켓, 패딩, 운동화 등 아이템이 자주 언급되었으며, 화이트, 그레이, 블랙 등 무난한 색상이 선호됩니다. 울, 퍼, 면 소재가 주로 사용되어 보온성과 스타일을 동시에 잡는 것이 핵심입니다.

---

## 추천 아이템

### 1. 니트
- **추천 이유:** 부드럽고 따뜻하며, 다양한 스타일링이 가능
- **추천 상품:** [화이트 울 니트](https://search.shopping.naver.com/search/all?query=화이트+울+니트)

### 2. 코트
- **추천 이유:** 깔끔한 실루엣과 보온성, 세련된 분위기 연출
- **추천 상품:** [그레이 울 코트](https://search.shopping.naver.com/search/all?query=그레이+울+코트)

### 3. 자켓
- **추천 이유:** 격식 있는 자리와 캐주얼 모두 소화 가능
- **추천 상품:** [블랙 가죽 자켓](https://search.shopping.naver.com/search/all?query=블랙+가죽+자켓)

### 4. 패딩
- **추천 이유:** 보온성을 높이면서도 스타일리시하게 연출 가능
- **추천 상품:** [크림 다운 패딩](https://search.shopping.naver.com/search/all?query=크림+다운+패딩)

### 5. 운동화
- **추천 이유:** 편안함과 캐주얼한 무드
- **추천 상품:** [화이트 운동화](https://search.shopping.naver.com/search/all?query=화이트+운동화)

---

## 피해야 될 소재
- **너무 두꺼운 퍼 또는 가죽 소재의 아우터:** 무거운 느낌과 부자연스러움
- **반짝이는 소재의 의상:** 지나치게 캐주얼하거나 포멀에 어울리지 않을 수 있음
- **과한 패턴 또는 화려한 디자인:** 깔끔한 오피스룩에 어울리지 않음

---

## 코디맵 예시

| 아이템       | 추천 색상 | 스타일 포인트                                   | 링크 예시                                              |
|--------------|------------|------------------------------------------------|--------------------------------------------------------|
| 니트         | 화이트, 그레이 | 깔끔하고 세련된 기본 니트, 오피스에 적합             | [화이트 울 니트](https://search.shopping.naver.com/search/all?query=화이트+울+니트) |
| 코트         | 그레이 또는 블랙 | 미니멀한 디자인, 슬림핏 또는 오버사이즈 추천        | [그레이 울 코트](https://search.shopping.naver.com/search/all?query=그레이+울+코트) |
| 자켓 또는 블레이저 | 블랙 또는 네이비 | 포멀한 느낌, 셔츠 또는 블라우스와 매치             | [블랙 가죽 자켓](https://search.shopping.naver.com/search/all?query=블랙+가죽+자켓) |
| 하의         | 슬림핏 슬랙스 또는 치노팬츠 | 깔끔한 실루엣, 오피스룩에 적합                     | [그레이 슬랙스](https://search.shopping.naver.com/search/all?query=그레이+슬랙스) |
| 신발         | 화이트 운동화 | 캐주얼하면서도 깔끔한 마무리                        | [화이트 운동화](https://search.shopping.naver.com/search/all?query=화이트+운동화) |

---

## 결론
맑은 겨울 출근 날에는 깔끔한 니트와 세련된 코트, 가죽 자켓, 그리고 편안한 운동화로 스타일과 보온성을 모두 챙기세요. 무난한 색상과 울, 퍼, 면 소재를 활용하면 자연스럽고 세련된 오피스룩을 완성할 수 있습니다.

---

**추가 참고 링크:**  
- [네이버 블로그 패션 스타일링 참고](https://m.blog.naver.com/purplemodern/224051226513)  
- [겨울 출근 코디 아이디어](https://m.blog.naver.com/serrri/223250900739)

```
토큰 사용량
입력 토큰 | 4668 | US$0.0009336
출력 토큰 | 975 | US$0.0007800000000000001
총 토큰 | 5643 | US$0.0017136
```

In [4]:
print(response[0], "\n")
print(response[1], "\n")
print(response[2], "\n")
print(response[3])

```검색 키워드 >> 겨울 맑은 날 출근 코디```

# 겨울 맑은 날 출근 코디 추천

네이버 블로그 내용을 종합하면, 겨울철 출근 패션은 따뜻함과 세련됨을 동시에 고려하는 것이 중요합니다. 특히, 니트, 코트, 자켓, 패딩, 운동화 등 아이템이 자주 언급되었으며, 화이트, 그레이, 블랙 등 무난한 색상이 선호됩니다. 울, 퍼, 면 소재가 주로 사용되어 보온성과 스타일을 동시에 잡는 것이 핵심입니다.

---

## 추천 아이템

### 1. 니트
- **추천 이유:** 부드럽고 따뜻하며, 다양한 스타일링이 가능
- **추천 상품:** [화이트 울 니트](https://search.shopping.naver.com/search/all?query=화이트+울+니트)

### 2. 코트
- **추천 이유:** 깔끔한 실루엣과 보온성, 세련된 분위기 연출
- **추천 상품:** [그레이 울 코트](https://search.shopping.naver.com/search/all?query=그레이+울+코트)

### 3. 자켓
- **추천 이유:** 격식 있는 자리와 캐주얼 모두 소화 가능
- **추천 상품:** [블랙 가죽 자켓](https://search.shopping.naver.com/search/all?query=블랙+가죽+자켓)

### 4. 패딩
- **추천 이유:** 보온성을 높이면서도 스타일리시하게 연출 가능
- **추천 상품:** [크림 다운 패딩](https://search.shopping.naver.com/search/all?query=크림+다운+패딩)

### 5. 운동화
- **추천 이유:** 편안함과 캐주얼한 무드
- **추천 상품:** [화이트 운동화](https://search.shopping.naver.com/search/all?query=화이트+운동화)

---

## 피해야 될 소재
- **너무 두꺼운 퍼 또는 가죽 소재의 아우터:** 무거운 느낌과 부자연스러움
- **반짝이는 소재의 의상:** 지나치게 캐주얼하거나 포멀에 어울리지

In [5]:
print(f"{response[1].index[0]} ({response[1].iloc[0]} 회)")

니트 (550 회)
